In [1]:
import wave
import struct
import matplotlib.pyplot as plt
import numpy as np

#we have whitecrown sparrow and american robin samples

class_ids = {
    'sparrow': 0,
    'robin': 1,
}


robin = wave.open('./recordings/ml-american-robin.wav', 'r')
frame_n = robin.getnframes()


In [25]:
# find silent frames of file
#
# @arg w sound wave file
# @arg threshold threshold value for sound intensity
#
# @return s_frame_val all silent frames
# @return frame_val all significant frames
# @sig_frames number of sig frames
# @frames total number of frames

def silent_frames(w,threshold, x=0, y=-1, verbose=False):
    #reset pointer in file
    w.rewind()
   
    #init frame arrayws
    init_size = w.getnframes()
    
    frames = np.empty(init_size)
    s_frames = np.empty(init_size)
    
    frames_n=0
    s_frames_n = 0
    
    if y == -1:
        y = init_size
    
    
    
    for i in range(w.getnframes()):
        # read a single frame and advance to next frame
        current_frame= w.readframes(1)
        
        if w.tell() >= x:
            #print(w.tell())

            # check for silence
            silent = True
            # wave frame samples are stored in little endian**
            # this example works for a single channel 16-bit per sample encoding
            unpacked_signed_value = struct.unpack("<h", current_frame) # *
            if abs(unpacked_signed_value[0]) > threshold:
                silent = False

            if silent:
                s_frames[s_frames_n] = w.tell()
                s_frames_n +=1
                if verbose:
                    print ("Frame %s is silent." % w.tell())
                    print ("silence found at second %s" % (w.tell()/w.getframerate()))
            else:
                frames[frames_n] = w.tell()
                frames_n +=1
                if verbose:
                    print ("Frame %s is not silent." % w.tell())
                
            if w.tell() == y:
                break

    frames = np.resize(frames,frames_n)
    s_frames = np.resize(s_frames,s_frames_n)
    
    return frames, s_frames        

In [6]:
frames, s_frames = silent_frames(robin, 50, verbose=False)

2536701


### 2003 pixels on png == 44100 frames on audio file

In [27]:
print(frames[::])
print(s_frames[::])

[1.692000e+03 1.693000e+03 1.694000e+03 ... 4.704835e+06 4.705076e+06
 4.705077e+06]
[1.000000e+00 2.000000e+00 3.000000e+00 ... 4.726654e+06 4.726655e+06
 4.726656e+06]


In [8]:
print(len(frames))
print(len(s_frames))

2536701
2189955


In [10]:
def frame_to_pixel(f):
    return f * 2003 / 44100

In [11]:
[frame_to_pixel(x) for x in[220000, 244200]]

[9992.290249433107, 11091.442176870749]

In [81]:
#TODO: fix method, something is wrong
# find silent frames of file using sliding window and ratio of heard frames to window size
#
# @arg w sound wave file
# @arg frames list of heard frames
# @arg stride how far the window moves each iter
# @arg tol ratio of heard/silent calls considered acceptable
#
# @return ranges 2d numpy array containing list of ranges


def find_audio(w, frames, window_size=150, stride=1, tol=0.3):
    left_f = 0
    right_f = window_size
    
    frame_end = int(frames[len(frames)-1])
    
    r_index = 0
    l_index = 0
    
    #values to return for range
    left = -1
    right = -1
    
    ranges = np.zeros((w.getnframes()), dtype=(float,2))
    
    #get inital heard values
    heard = 0
    
    for f in frames:
            #outside window range set new values
            if f >= right_f:
                #set next window  ind 
                left_f += 1
                right_f += 1
                
                #set index in frames
                r_index = heard
                l_index += 1
                
                break
            #increment heard frames
            heard += 1
            
    #ratio of heard to window size
    ratio = heard/window_size
    
    #if tol is high set left range
    if ratio >= tol:
        left = left_f
        
    i = 0
    
    while(right_f < frame_end):
        #check if left value in frame range
        if frames[l_index] <= left_f:
            l_index += 1
            heard -= 1
            #print('left out of range')
    
    
        #check if right value in frame range
        if frames[r_index] <= right_f:
            r_index += 1
            heard += 1
            #print('right in range')        
            
            
        #print(ratio)
            
        ratio = heard/window_size
        
        if ratio >= tol and left == -1:
            left = left_f
            
        #print(ratio , '' , tol)

        
            
        if ratio < tol and left != -1:
            right = right_f
            ranges[i] = [left,right]
            i +=1
            left = -1
        
        right_f += 1
        left_f += 1
        
    ranges = np.resize(ranges,(i,2))        
   
    return ranges


In [167]:
def get_freq(w):
    #reset pointer in file
    w.rewind()
    
    init_size = w.getnframes()
    
    freqs = np.zeros(init_size)
    
    ind = 0
    
    #grab frequencies
    for i in range(w.getnframes()):
        # read a single frame and advance to next frame
        current_frame= w.readframes(1)
        freq = abs(struct.unpack("<h", current_frame)[0])
        freqs[i] = freq
        i+=1


    return freqs



#find audio using song filter method
def song_filter(freqs, window_size=22, stride=1, tol=0.3):
    
    #window bounds
    l,r = 0,0
    
    #range of sound
    l_range,r_range = -1,-1
    
    #measures for tol
    sum_n = 0
    max_n = -1
    
    
    #init frame arrayws
    init_size = len(freq)
    
    ranges = np.zeros((init_size), dtype=(float,2))
    
      
    #set init values
    for i in range(0,window_size-1):
        sum_n += freqs[i]
        if max_n < freqs[i]:
            max_n = freqs[i]
        r+=1
        
    if sum_n > 0:    
        ratio = max_n/sum_n
    
        if ratio >= tol:
            l_range=l
            r_range=r
            
    l+=1
    r+=1
    
    index = 0
    
    while(r < init_size):
        
        #need to loop over window to find new max
        if(freqs[l-1]==max_n):
            max_n=-1
            for i in range(l,r):
                if max_n < freqs[i]:
                    max_n = freqs[i]
                    
        #compare max to new r
        if(freqs[r-1]==max_n):
            if(freqs[r] > freqs[r-1]):
                max_n = freqs[r]
    
        sum_n -= freqs[l-1]
        sum_n += freqs[r]
    
        #calculate ratio
        ratio = max_n / sum_n
        
        #set l if not already
        if ratio >= tol:            
            if l_range == -1:
                l_range = l
            
            #check if l is set then set r
        else:
            if l_range >=0:
                r_range = r-1
                ranges[index] = [l_range,r_range]   
    
        #slide window
        r+=1
        l+=1

    ranges = np.resize(ranges,(i,2))   
    
    return(ranges)      

    

In [160]:
freq = get_freq(robin)

In [168]:
ranges_SF = song_filter(freq)


/home/dalbosta/anaconda3/envs/conda_env/lib/python3.7/site-packages/ipykernel_launcher.py:81: RuntimeWarning: invalid value encountered in double_scalars


In [169]:
ranges_SF

array([[1.340000e+03, 4.726654e+06],
       [0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00],
       ...,
       [0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00]])

In [40]:
#grabs audio ranges for a given wav file
# @arg w wave file
# @arg t1 frequency tol for detecting noise
# @arg t2 heard/windowsize ratio tolerance
# @arg window_size size of sliding window
# @arg stride stride of window
#
# @return 2d array of audio ranges containing sound
def audio_ranges(w,t1,t2, window_size=150, stride=1):
    frames, s_frames = silent_frames(w,t1)
    return(find_audio(robin,test,tol = t2))

In [41]:
test = audio_ranges(robin,50,0.2)

4455


array([[1.703000e+03, 1.853000e+03],
       [1.922000e+03, 2.072000e+03],
       [1.941000e+03, 2.091000e+03],
       ...,
       [4.704289e+06, 4.704439e+06],
       [4.704565e+06, 4.704715e+06],
       [4.704760e+06, 4.704910e+06]])